### Train an MMPose Network
- note that I am using the conda env at /n/groups/datta/tim_sainburg/conda_envs/openmmlab

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from pathlib import Path
import sys
from datetime import datetime

In [3]:
pretrained_model_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models")

### Download checkpoint for a pretrained model (if desired)
Alternatively, use a previous mouse model as a pretrained model

In [4]:
pretrained_model_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models")
pretrained_model_directory.mkdir(parents=True, exist_ok=True)

In [5]:
# find models here: https://github.com/open-mmlab/mmpose/tree/main/configs
pretrain_model = "rtmpose-m_8xb64-210e_ap10k-256x256"

In [6]:
command = f"source activate {Path(sys.executable).parents[1]}; mim download mmpose --config {pretrain_model} --dest {pretrained_model_directory.as_posix()}"
print(command)

source activate /n/groups/datta/tim_sainburg/conda_envs/openmmlab; mim download mmpose --config rtmpose-m_8xb64-210e_ap10k-256x256 --dest /n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models


In [7]:
!{command}

/bin/bash: activate: No such file or directory
processing rtmpose-m_8xb64-210e_ap10k-256x256...
rtmpose-m_simcc-ap10k_pt-aic-coco_210e-256x256-7a041aa1_20230206.pth exists in /n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models
Traceback (most recent call last):
  File "/n/groups/datta/tim_sainburg/conda_envs/openmmlab/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/n/groups/datta/tim_sainburg/conda_envs/openmmlab/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/n/groups/datta/tim_sainburg/conda_envs/openmmlab/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/n/groups/datta/tim_sainburg/conda_envs/openmmlab/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/n/groups/datta/tim_sainburg/conda_envs/openmmlab/lib/python3.10/site-packages/click/core.py", line 1434, in in

In [8]:
!ls {pretrained_model_directory.as_posix()}

rtmdet_s_8xb32-300e_coco_20220905_161602-387a891e.pth
rtmdet_s_8xb32-300e_coco.py
rtmpose-m_8xb64-210e_ap10k-256x256.py
rtmpose-m_simcc-ap10k_pt-aic-coco_210e-256x256-7a041aa1_20230206.pth


In [9]:
pretrained_model_directory

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/pretrained_mm_models')

### Parameters and dataset

In [10]:
model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256'

# Where the COCO format dataset is located (created in the previous notebook)
# dataset_directory = Path("/n/groups/datta/tim_sainburg/projects/24-04-02-neuropixels-chronic/data/keypoints/coco-trainingsets/240408-mmpose-multianimal-chronic_v3/")
dataset_directory = Path("/n/groups/datta/6cam_keypoint_networks/training_data/JP_CW_scale_annos/COCO_format/")

# this file contains info about the dataset (keypoints, skeleton, etc) needed for traiing
# dataset_info_loc =  Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/skeletons/sainburg25pt.py")
# n_keypoints = 25
dataset_info_loc = Path("/n/groups/datta/Jonah/Local_code_groups/6cam_repos/multicam_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/skeletons/weinreb15pt.py")
n_keypoints = 15

# which config to use (this is what we base the config off of). Should be in the mmpose repo. 
config_loc = Path('/n/groups/datta/tim_sainburg/projects/mmpose/configs/animal_2d_keypoint/rtmpose/ap10k/rtmpose-m_8xb64-210e_ap10k-256x256.py')

# which pretrained model to use (point to .pth file). Pretrained model should be the same model architecture. 
pretrained_model = pretrained_model_directory / "rtmpose-m_simcc-ap10k_pt-aic-coco_210e-256x256-7a041aa1_20230206.pth"
# pretrained_model = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-01-05-13-46-05_748568/best_PCK_epoch_230.pth'
use_pretrained_model = True

# working directory (where model output is saved)
# output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/mm_training")
output_directory = Path("/n/groups/datta/6cam_keypoint_networks/mm_pose/Jonah/20241030_v1")
formatted_datetime = datetime.now().strftime("%y-%m-%d-%H-%M-%S")
working_directory = (output_directory / 'rtmpose' / f"{model_name}_{formatted_datetime}")
working_directory.mkdir(parents=True, exist_ok=True)

In [11]:
assert config_loc.exists()
assert dataset_directory.exists()

### Register the new dataset

In [12]:
from mmpose.registry import DATASETS
from mmpose.datasets.datasets.base import BaseCocoStyleDataset
dataset_type = 'CoCo15pt'  # must match class name
@DATASETS.register_module()
class CoCo15pt(BaseCocoStyleDataset):
    METAINFO: dict = dict(from_file=dataset_info_loc)

# You shouldn't need to change anything below here
(unless you are using a different skeleton model)

### Display compute / environment info (for future reference)

In [13]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


gcc (GCC) 6.2.0
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [14]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env
import sys
import mmdet
import torch
import torchvision
import mmpose
from mmcv.ops import get_compiling_cuda_version, get_compiler_version

def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info

print(f"Environment: {sys.executable}")
for name, val in collect_env().items():
    print(f'{name}: {val}')
# Check Pytorch installation
print('cuda version:', get_compiling_cuda_version())
print('compiler information:', get_compiler_version())
print('torch version:', torch.__version__, torch.cuda.is_available())
print('torchvision version:', torchvision.__version__)
print('mmpose version:', mmpose.__version__) 

Environment: /n/groups/datta/tim_sainburg/conda_envs/openmmlab/bin/python
sys.platform: linux
Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
CUDA available: True
numpy_random_seed: 2147483648
GPU 0: NVIDIA A100 80GB PCIe MIG 3g.40gb
CUDA_HOME: /n/groups/datta/tim_sainburg/conda_envs/openmmlab
NVCC: Cuda compilation tools, release 12.1, V12.1.105
GCC: gcc (GCC) 6.2.0
PyTorch: 2.1.0
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute

### Create config file

In [15]:
from mmengine import Config

In [16]:
cfg = Config.fromfile(config_loc.as_posix())

In [17]:
# load COCO pre-trained weight
if use_pretrained_model:
    cfg.load_from = pretrained_model.as_posix()

In [18]:
# set the dataset directory
cfg.data_root = dataset_directory.as_posix()

# set the working directory
cfg.work_dir = working_directory.as_posix()
cfg.randomness = dict(seed=0)

In [19]:
# set dataset configs
cfg.dataset_type = dataset_type
cfg.data_mode = 'topdown'

# number of keypoints
cfg.model.head.out_channels = n_keypoints

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.ann_file = 'annotations/instances_train.json'
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img='train/')


cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.bbox_file = None
cfg.val_dataloader.dataset.ann_file = 'annotations/instances_val.json'
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img='val/')

cfg.test_dataloader.dataset.type = cfg.dataset_type
cfg.test_dataloader.dataset.bbox_file = None
cfg.test_dataloader.dataset.ann_file = 'annotations/instances_val.json'
cfg.test_dataloader.dataset.data_root = cfg.data_root
cfg.test_dataloader.dataset.data_prefix = dict(img='val/')

# set to custom datset
cfg.train_dataloader.dataset.metainfo = dict(from_file=dataset_info_loc.as_posix())
cfg.val_dataloader.dataset.metainfo = dict(from_file=dataset_info_loc.as_posix())
cfg.test_dataloader.dataset.metainfo = dict(from_file=dataset_info_loc.as_posix())

# set evaluator
cfg.val_evaluator = dict(type='PCKAccuracy')
cfg.test_evaluator = cfg.val_evaluator

cfg.default_hooks.checkpoint.save_best = 'PCK'
cfg.default_hooks.checkpoint.max_keep_ckpts = 15
cfg.default_hooks.checkpoint.interval = 10

cfg.max_epochs = 2000
cfg.train_cfg.max_epochs = 2000

In [20]:
print(cfg)

Config (path: /n/groups/datta/tim_sainburg/projects/mmpose/configs/animal_2d_keypoint/rtmpose/ap10k/rtmpose-m_8xb64-210e_ap10k-256x256.py): {'default_scope': 'mmpose', 'default_hooks': {'timer': {'type': 'IterTimerHook'}, 'logger': {'type': 'LoggerHook', 'interval': 50}, 'param_scheduler': {'type': 'ParamSchedulerHook'}, 'checkpoint': {'type': 'CheckpointHook', 'interval': 10, 'save_best': 'PCK', 'rule': 'greater', 'max_keep_ckpts': 15}, 'sampler_seed': {'type': 'DistSamplerSeedHook'}, 'visualization': {'type': 'PoseVisualizationHook', 'enable': False}, 'badcase': {'type': 'BadCaseAnalysisHook', 'enable': False, 'out_dir': 'badcase', 'metric_type': 'loss', 'badcase_thr': 5}}, 'custom_hooks': [{'type': 'EMAHook', 'ema_type': 'ExpMomentumEMA', 'momentum': 0.0002, 'update_buffers': True, 'priority': 49}, {'type': 'mmdet.PipelineSwitchHook', 'switch_epoch': 180, 'switch_pipeline': [{'type': 'LoadImage', 'backend_args': {'backend': 'local'}}, {'type': 'GetBBoxCenterScale'}, {'type': 'Random

In [21]:
# set preprocess configs to model
cfg.model.setdefault('data_preprocessor', cfg.get('preprocess_cfg', {}))

{'type': 'PoseDataPreprocessor',
 'mean': [123.675, 116.28, 103.53],
 'std': [58.395, 57.12, 57.375],
 'bgr_to_rgb': True}

In [22]:
# save configuration file for future reference
cfg.dump(working_directory / 'config.py')

### run network

In [23]:
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

In [24]:
# build the runner from config
runner = Runner.from_cfg(cfg)

11/01 20:37:12 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: NVIDIA A100 80GB PCIe MIG 3g.40gb
    CUDA_HOME: /n/groups/datta/tim_sainburg/conda_envs/openmmlab
    NVCC: Cuda compilation tools, release 12.1, V12.1.105
    GCC: gcc (GCC) 6.2.0
    PyTorch: 2.1.0
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compu

In [25]:
# start training
runner.train()

/n/groups/datta/tim_sainburg/conda_envs/openmmlab/lib/python3.10/site-packages/mmpose/datasets/transforms/common_transforms.py:656: UserWarning: Blur is not pixel-level transformations. Please use with caution.
  warnings.warn(
/n/groups/datta/tim_sainburg/conda_envs/openmmlab/lib/python3.10/site-packages/mmpose/datasets/transforms/common_transforms.py:656: UserWarning: MedianBlur is not pixel-level transformations. Please use with caution.
  warnings.warn(
/n/groups/datta/tim_sainburg/conda_envs/openmmlab/lib/python3.10/site-packages/mmpose/datasets/transforms/common_transforms.py:656: UserWarning: CoarseDropout is not pixel-level transformations. Please use with caution.
  warnings.warn(


loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stem.0.bn.weight:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stem.0.bn.bias:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stem.1.bn.weight:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stem.1.bn.bias:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stem.2.bn.weight:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stem.2.bn.bias:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stage1.0.bn.weight:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stage1.0.bn.bias:weight_decay=0.0
11/01 20:37:20 - mmengine - INFO - paramwise_options -- backbone.stage1.1.main_conv.bn.weight:weight_decay=0.0
11/01 20:37:20 - m

### The config and path for running inference will be in the working directory

In [1]:
2+2

4